Suppose you have some custom modules, named pipeline and external_fns, and want to use them on a coiled cluster.  And, further suppose that these are contained in folders within your current working directory for Python. 

These will work automatically with a Local.Cluster, as your system already knows to look for them in the current active directory.  However, for these functions to work on a Coiled Cluster, the Dask workers need to know how to find those modules.  

The following sample code shows how to do this.

Code based on input from Kelsey Skvoretz (https://github.com/skvorekn; https://github.com/skvorekn/repr-coiled-upl-dir) and James Bourbeau (https://github.com/jrbourbeau)

In [1]:
# Imports you will need
# In addition to your custom modules, your current environment will need to include 
# dask & distributed, with all of their depenencies.  These will already be in a 
# coiled default environment.

# Standard Packages
import os

# Specialty Packages
from dask.distributed import Client, LocalCluster
from distributed.diagnostics.plugin import UploadDirectory

# Coiled
import coiled

# Your custom modules.
from external_fns.misc import get_prefix
from pipeline.functions.item_level import runner

In [2]:
# Create a Cluster

get_prefix()

cluster = coiled.Cluster(
            name='upload-directory-test',
            n_workers=1,
            worker_cpu=1,
            worker_class='distributed.Nanny'  #  Need to test if this is required.
        )
client = Client(cluster)

client.wait_for_workers(n_workers=1)

print("Created client")

Output()

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-greg-sm27-100379-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-greg-sm27-100379-firewall -> coiled-dask-greg-sm27-100379-firewall]
Created FW rules: coiled-dask-greg-sm27-100379-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-greg-sm27-100379-cluster-firewall -> coiled-dask-greg-sm27-100379-cluster-firewall]
Created scheduler VM: coiled-dask-greg-sm27-100379-scheduler (type: t3.medium, ip: ['44.201.46.87'])


C:\Users\greg\anaconda3\envs\shakedown-39\lib\site-packages\distributed\client.py:1096: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| dask        | 2021.12.0 | 2022.01.0 | None    |
| distributed | 2021.12.0 | 2022.01.0 | None    |
| msgpack     | 1.0.2     | 1.0.3     | None    |
+-------------+-----------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Created client


In [3]:
# Function to update paths on workers & code to upload modules

def update_path(dask_worker):
        import pathlib
        import sys
        path = str(pathlib.Path(dask_worker.local_directory).parent)
        if path not in sys.path:
            sys.path.insert(0, path)

client.run(update_path)

plugin = UploadDirectory('pipeline', update_path=False, restart=False)
client.register_worker_plugin(plugin) 

print("Client Path Updated")

Client Path Updated


In [4]:
# See what the directory structure looks like
def test_func():
    dirs = []
    for d in os.walk('dask-worker-space'):
        dirs.append(d)
    return dirs

job = client.submit(test_func)
print(job.result())

# Example output:
# [
#     (
#         'dask-worker-space',
#         ['worker-pvqyc2yh', 'pipeline'],
#         ['worker-pvqyc2yh.dirlock', 'purge.lock', 'global.lock']
#     ),
#     ('dask-worker-space/worker-pvqyc2yh', ['storage'], []),
#     ('dask-worker-space/worker-pvqyc2yh/storage', [], []),
#     ('dask-worker-space/pipeline', ['functions'], ['__init__.py', 'errors.py']),
#     ('dask-worker-space/pipeline/functions', [], ['__init__.py', 'item_level.py'])
# ]

[('dask-worker-space', ['pipeline', 'worker-_s40l6lt'], ['purge.lock', 'worker-_s40l6lt.dirlock', 'global.lock']), ('dask-worker-space/pipeline', ['functions'], ['__init__.py', 'errors.py']), ('dask-worker-space/pipeline/functions', [], ['__init__.py', 'item_level.py']), ('dask-worker-space/worker-_s40l6lt', ['storage'], []), ('dask-worker-space/worker-_s40l6lt/storage', [], [])]


In [5]:
# Show that all works. 
runner(client)

['test', 'list']
['test', 'list']
['test', 'list']
['test', 'list']
['test', 'list']


In [6]:
# Clean up

client.close()
cluster.close()

Cluster deleted successfully.